In [1]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import wikipedia as wiki
import re
from nltk.corpus import wordnet as wn
from itertools import product

C:\Users\Сергей\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
%%time
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
glove_model.init_sims(replace=True)

Wall time: 2min 7s


In [5]:
def wn_similarity(x, y):
    for sense1, sense2 in product(x, y):
        d = wn.wup_similarity(sense1, sense2)
        if d != None:
            wn_sim = d
        else:
            wn_sim = 0.1
    return wn_sim

In [6]:
%%time
dataset = open("validation.txt", encoding = 'utf-8').readlines()
X, y, z = [], [], 0
for d in range(len(dataset)):
    try:
        lin = dataset[d].strip().split(',')
            
        target_sim = glove_model.similarity(lin[0], lin[2])
        comp_sim = glove_model.similarity(lin[1], lin[2])
        targ_comp_sim = glove_model.similarity(lin[0], lin[1])
            
        try:
            sum_lin_0 = wiki.summary(lin[0])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_0 = wiki.summary(e.options[0])
        try:
            sum_lin_1 = wiki.summary(lin[1])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_1 = wiki.summary(e.options[0])
        lin_0_wiki = len(re.findall(lin[2], sum_lin_0))
        lin_1_wiki = len(re.findall(lin[2], sum_lin_1))
        
        syns0 = wn.synsets(lin[0])
        syns1 = wn.synsets(lin[1])
        syns2 = wn.synsets(lin[2])
        first_wn = wn_similarity(syns0, syns2)
        second_wn = wn_similarity(syns1, syns2)
        between_themselves_wn = wn_similarity(syns0, syns1)
                
        X.append((target_sim, comp_sim, target_sim-comp_sim, targ_comp_sim, lin_0_wiki, lin_1_wiki, lin_0_wiki-lin_1_wiki, first_wn, second_wn, between_themselves_wn, first_wn-second_wn))
        y.append(lin[3])
        z += 1
        print("processed lines: " + str(z))
    except:
        continue

processed lines: 1
processed lines: 2
processed lines: 3


C:\Users\Сергей\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


processed lines: 4
processed lines: 5
processed lines: 6
processed lines: 7
processed lines: 8
processed lines: 9
processed lines: 10
processed lines: 11
processed lines: 12
processed lines: 13
processed lines: 14
processed lines: 15
processed lines: 16
processed lines: 17
processed lines: 18
processed lines: 19
processed lines: 20
processed lines: 21
processed lines: 22
processed lines: 23
processed lines: 24
processed lines: 25
processed lines: 26
processed lines: 27
processed lines: 28
processed lines: 29
processed lines: 30
processed lines: 31
processed lines: 32
processed lines: 33
processed lines: 34
processed lines: 35
processed lines: 36
processed lines: 37
processed lines: 38
processed lines: 39
processed lines: 40
processed lines: 41
processed lines: 42
processed lines: 43
processed lines: 44
processed lines: 45
processed lines: 46
processed lines: 47
processed lines: 48
processed lines: 49
processed lines: 50
processed lines: 51
processed lines: 52
processed lines: 53
proces

In [7]:
print(len(X))

2691


In [8]:
X_train = X[0:1691]
y_train = y[0:1691]
X_test = X[1691:]
y_test = y[1691:]

In [14]:
neigh = KNeighborsClassifier(n_neighbors=20, p=1)
neigh.fit(X_train, y_train)

predictions = neigh.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.68      0.66      0.67       508
          1       0.65      0.67      0.66       492

avg / total       0.67      0.67      0.67      1000



In [26]:
ran_for = RandomForestClassifier(n_estimators=15, random_state=42,
                                 max_features=None, max_depth=5)
ran_for.fit(X_train, y_train)

predictions = ran_for.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.74      0.62      0.68       508
          1       0.66      0.77      0.72       492

avg / total       0.70      0.70      0.70      1000

